#User Data Simulation
In this part, we aim to generate customer purchase record needed to build recommender engine. To generate such data, I assume there exist a transitive relation among designer-product-customer, which means I assume that the characteristic group appear in designer will also appear in the customer. I make this assumption based on the rule of supply and demand. If there is a group of designer focus on selling brooks, necklase or bracelat, there must exist a specific group of customer that consume such product.<br>


To achieve this goal. we have to split customers into active and non-active customers based on 20-80 rule. 20% is active user while the reamining is non-active user. In both group, we further divide customers based on cluster of designer. The numnber of each cluster is propotional to the size of cluster calculated in ExploratoryAnalysis-Designer.<br><br> The only difference between two groups is that the active customer shows stronger group characterstic. For example, if a customer is both active user and a bracelet/brooks/necklase lover, then bracelet/brooks/necklase product will account for,say, 65% percent of her prodcut while the remainging 35% would be randomly chosen from all products. This characteristic is implemented by the weight alpha in UserDicGenerator.<br><br>

Here we implement two functions, one is UserDataGenerator and the other is UserDicGenerator. The former focus on split the customers into active and non-active and decide how many customers should be in a specific group. The latter focus on deciding how many percent of the purchase represent the group characteristic of a customer.

In [2]:
import pandas as pd
import numpy as np

In [45]:
Data=pd.read_csv('Intermediate/data_final.csv',sep=",",encoding='utf8',low_memory=False)
UserData=Data[['category','tid','subcategory','Product Tag','owner']]

In [12]:
UserData.head()

,category,tid,subcategory,Product Tag,owner
0,3,1fAYZRbb,紙膠帶,紙膠帶,yuhuirustudio
1,10,1Wpbjtdt,糖果/軟糖,"乾果醬,交換禮物,繽紛,聖誕節",doggysweets
2,1,1cf23LXr,手提包,手提帶,ningoo
3,2,1XkKaVA0,髮飾,"好煩小姐,髮帶,點點",mshowfan
4,2,1cDqf1tY,髮飾,"好煩小姐,復古,手作,混織,髮帶",mshowfan


In [4]:
g_num=list(pd.read_csv("Intermediate/group_num.csv",sep=",",encoding='utf8',header=None)[0])
g_ratio=[float(e)/float(sum(g_num)) for e in g_num]
criteria=pd.read_csv("Intermediate/criteria.csv",sep=",",encoding='utf8')

This table is the group characteris of each group, which is generated according to the ExploratoryAnalysis-Designer. The higher the column index, the less representative it is. For example, for the group 3, from the most representative to less is 杯墊->圍兜/領巾/配件->髮飾->耳環-> 	捲線器/電線收納

In [14]:
criteria

,0,1,2,3,4
0,iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,電腦包
1,媽媽包,啟發玩具,童鞋,童裝,彌月收涎
2,杯墊,圍兜/領巾/配件,髮飾,耳環,捲線器/電線收納
3,皮夾/錢包,束口後背包,後背包,零錢包,手提包
4,胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環
5,外套/大衣 (女),項鍊/墜子,裙子,針織衫/毛衣 (女),襯衫 (女)
6,串珠材料,手鍊/手環,胸針,戒指,項鍊/墜子
7,貼紙,筆記本/手帳,化妝包/雜物包,卡片/明信片,杯墊
8,杯墊,鑰匙圈/鑰匙包,Android 週邊,票夾/隨身夾,咖啡杯/馬克杯
9,iPhone 週邊,手提包,上衣 (女),T 恤 (女),上衣/T 恤 (男)


In [47]:
def UserDataGenerator(num):
    #Split the customers into two groups： 20％ active and 80％ non active
    act_num=0.2*num
    non_act_num=0.8*num

    #Randomly choose customer's ID to be active customer, the remaining ID are non active customers
    act_user=np.random.choice(num,act_num,replace=False)
    non_act_user=list(set(range(num)).difference(set(act_user)))
    
    Userdic_act=[]
    Userdic_nonact=[]
    for g in range(len(g_ratio)-1):
        #The number of g_size_act, g_size_nonact determine how many customers in the active and non-active respectively
        g_size_act=int(g_ratio[g]*act_num) 
        g_size_nonact=int(g_ratio[g]*non_act_num)
        
        #choose the numnber of customer according to the size
        g_user_act=act_user[0:g_size_act]
        g_user_nonact=non_act_user[0:g_size_nonact]
        
        #Delete the customer ID which have been chosen from the original list
        act_user=[e for e in act_user if e not in g_user_act]
        non_act_user=[e for e in non_act_user if e not in g_user_nonact]
        
        #Generate the customer data according to following dictionary fomat:
        #{'UID':XXX 'Subcategory','subcategory':XXX,'Product Tag':XXX,'tid':xxxxxxx,'designer':xxxxxx}
        #Alpha is the strength of tendency whether a user tend to represent the character of group, see UserDicGenerator
        #The alpha of the active can be arbitary chosen from 0.5~0.99 while 0.0~0.5 for the non-active
        #Here we use 0.6 for active and 0.3 for non active
        #print "test"
        Userdic_act.extend(UserDicGenerator(g_user_act,0.6,criteria.ix[g]))
        Userdic_nonact.extend(UserDicGenerator(g_user_nonact,0.3,criteria.ix[g]))
        
    return Userdic_act,Userdic_nonact

In [48]:
def UserDicGenerator(u_list,alpha,c):
    udic=[]
    for u in u_list:
        #Declare the dicionary used to store customer data
        dic={}
        dic['UID']=u
        dic['subcategory']=[]
        dic['Product Tag']=[]
        dic['tid']=[]
        dic['designer']=[]
        
        #alpha=0.6 is just randomly choose. this value is given to actvie user to represent it have more chance to represent the characteristiv
        #of group
        #We define the active customer have 10~20 purchase records while the non active customers are less than 10 
        if alpha==0.6:
            draw=np.random.randint(10,20)
        else:
            draw=np.random.randint(1,10)
        tmp_draw=draw
        tmp=0
        while tmp_draw>0:
            dic['subcategory'].append(c[tmp%len(c)])#pick product with first subcategory then second, third....
            choice=UserData[UserData['subcategory']==c[tmp%len(c)]].index
            purchase=UserData.ix[np.random.choice(choice,1),:]
            dic['tid'].extend(purchase['tid'])
            dic['Product Tag'].extend(purchase['Product Tag'])
            dic['designer'].extend(purchase['owner'])
            
            #if the customer already has alpha％ representative product, then the remaining 1-alpha ％ would be randomly choosen
            #here we add another 0.3 to add more variety, so the active customer can have 0.6~0.9 representative data while 0.3~0.6 for nonactive ones
            #The reason why I let non active customer have chance to buy more than half percent representative data is to add noise, which aims to
            #make the dataset more close to reality
            if tmp_draw<int(draw*(1.0-np.random.uniform(alpha,alpha+0.3))):
                choice=UserData.index
                purchase=UserData.ix[np.random.choice(choice,1),:]
                dic['subcategory'].extend(purchase['subcategory'])
                dic['tid'].extend(purchase['tid'])
                dic['Product Tag'].extend(purchase['Product Tag'])
                dic['designer'].extend(purchase['owner'])
            tmp+=1
            tmp_draw-=1
        udic.append(dic)
    return udic
    

In [77]:
#Generate the purchse record of 5000 unique user and combine together
d_act,d_nonact=UserDataGenerator(5000)
D=[]
D.extend(d_act)
D.extend(d_nonact)

In [78]:
#Convert each user data from dic to dataframe then concatenate
tmp_df=[pd.DataFrame(e) for e in D]
TmpUserData=pd.concat(tmp_df,axis=0)
TmpUserData.index=range(len(TmpUserData))

In [79]:
#Groupby features by UID
G_design=TmpUserData.groupby('UID')['designer'].apply(lambda x:",".join(x))
G_tid=TmpUserData.groupby('UID')['tid'].apply(lambda x:",".join(x))
G_sub=TmpUserData.groupby('UID')['subcategory'].apply(lambda x:",".join(x))
G_Pdtag=TmpUserData.groupby('UID')['Product Tag'].apply(lambda x:",".join(x))

UserDf=pd.DataFrame(data=np.transpose([range(len(G_design)),G_design.values,G_tid.values,G_sub.values,G_Pdtag.values]),columns=['UID','designer','tid','subcategory','Product Tag'])


In [52]:
UserDf.head()

,UID,designer,tid,subcategory,Product Tag
0,0,"decouart,beebipeace,yuni-handicraft,pixostyle,...","12hKzVrR,1Z9PLIkF,1uZFROja,1SCt77gk,1DiiGyna,1...","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,電腦包,手鍊/手環","花,女生,幾何,抽象,floral,green,white,samsungs5,samsun..."
1,1,"umade,jkzakka-tw,designsurfing,snupped,hellolu...","17pJo4WQ,1HKJoFtW,1RPxxBCt,115GBxFE,1FasZHpZ,1...","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,電腦包,手鍊/手...","htc,藝術創作,sony,插畫,handmade,海,攝影作品,攝影photography..."
2,2,"emmptyforest,davidloumang,dessin,pixostyle,gym...","1FA2Zm8S,1mPcr5qv,1A7nI2jr,1WEHFnos,1hFcOv55,1...","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,電腦包,iPho...","design,handmade,創意,客製化禮物,手作,手機殼,押花,禮物,設計,情人節,i..."
3,3,madebyflowers,1_N2iiua,iPhone 週邊,"design,簡約,禮物,禮品,手工,手作,情人節,客製化禮物,可愛,創意,handmade,設計"
4,4,"saykishish,domoreidea,eclotdesign,befine","1eyLzcRR,1tWuC8cp,1euEGmJr,1U6aDzXw","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套","復古,手工,手工皮革,皮革,簡約,手機包,萬用包,design,handmade,leath..."


In [80]:
UserDf['Content weight']=np.random.uniform(0,1,len(UserDf))
UserDf['Style weight']=1.0-UserDf['Content weight']

In [81]:
UserDf.head()

,UID,designer,tid,subcategory,Product Tag,Content weight,Style weight
0,0,"panicjunkie,beanbeancase,lovespringtime,innere...","1LNcMozq,1fKcWUPZ,1dWKrV5w,1NrPloT8,1xo784ZW,1...","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,電腦包,束口後背...","iPhone 週邊,6,plus,nexus5,nexus4,nexus,nexus6,ht...",0.050896,0.949104
1,1,"edie,googoods,wakakuwa,kimmidoll,wunghuh,daffo...","1R9IzDnu,149GTys_,18H5ZFLX,159QNvoZ,1z4JPC9Z,1...","胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,...","交換禮物,情人節禮物,生日禮物,禮物,不鏽鋼,つまみ細工,和風髮飾,手作,日式布花,日本花簪...",0.806415,0.193585
2,2,"deliatai,oone-n-only,tsukiniyorosiku,twine,ses...","1EB9FX2e,1MZaVziY,1YY6E82N,1TaTLdiX,14yhgn_L,1...","胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,...","小太出品,小小人,絢太你好,胸章,design,handmade,natural系列,手工,...",0.036503,0.963497
3,3,"iamidesign,littleprince,everythinginbetween,si...","14Rq3sEa,1VTZwwBQ,1xoV23DQ,1f1NhrOS,1XBJ0jUP","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,其他7","聖誕節,貓,中秋節,手機殼,iphone6,畢業,悠遊卡手機殼,情人節,悠遊卡保護殼,悠遊卡...",0.883987,0.116013
4,4,"bang-on-shop,pollinosis,lunablue,nadia,ishan13...","1xfaWs4D,1lwO72OY,1unlj5pu,1ukhY4Bu,1xHzNBML,1...","胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,...","design,handmade,natural系列,wood,獨一無二,白水晶,治療,透明,...",0.591378,0.408622


##Purify
Recall that have calculated the representative product tag in the ExploratoryAnalysis-Product by TF-IDF. We have to purify the rare product tag now 

In [82]:
Feature_p=pd.read_csv("Intermediate/feature_p.csv",sep=",",encoding='utf8')

In [83]:
Feature_p.head()

,Product Tag,Values
0,螢光筆,-0.000083
1,鍊子,-0.000090
2,平安,-0.000291
3,寬版洋裝,-0.000017
4,hand-painting,-0.000038


In [84]:
Feature_filter=list(Feature_p['Product Tag'])

In [85]:
for i in xrange(len(UserDf)):
    UserDf.ix[i,'Product Tag']=",".join(list(set(UserDf.ix[i,'Product Tag'].split(",")).intersection(set(Feature_filter))))
    if UserDf.ix[i,'Product Tag'] is np.nan:
        UserDf.ix[i,'Product Tag']=UserDf.ix[i,'subcategory']

## Adding some browsing records and new user
Here we define new user as an user who has an account but no pruchase record. We assume the number of new user is 0.05*number of old user

###Adding new user

In [86]:
p=0.05
new_num=p*len(UserDf)
User_new=pd.DataFrame(data=0,index=range(int(new_num)),columns=UserDf.columns)
UserDf=pd.concat([UserDf,User_new],axis=0)

In [87]:
UserDf.index=range(len(UserDf))

In [88]:
UserDf.head()

,UID,designer,tid,subcategory,Product Tag,Content weight,Style weight
0,0,"panicjunkie,beanbeancase,lovespringtime,innere...","1LNcMozq,1fKcWUPZ,1dWKrV5w,1NrPloT8,1xo784ZW,1...","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,電腦包,束口後背...","三星,google,htc,押花,nexus,note5,自然,特別,iPhone 週邊,禮...",0.050896,0.949104
1,1,"edie,googoods,wakakuwa,kimmidoll,wunghuh,daffo...","1R9IzDnu,149GTys_,18H5ZFLX,159QNvoZ,1z4JPC9Z,1...","胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,...","fashion,羊毛氈,日本花簪,項鍊,胸章,兒童,迷你小相本鑰匙圈◆幾何系列◆,desig...",0.806415,0.193585
2,2,"deliatai,oone-n-only,tsukiniyorosiku,twine,ses...","1EB9FX2e,1MZaVziY,1YY6E82N,1TaTLdiX,14yhgn_L,1...","胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,...","ses,胸章,鑰匙圈,創意,design,純銀項鍊,韓風髮箍,kkk.首飾りnecklace...",0.036503,0.963497
3,3,"iamidesign,littleprince,everythinginbetween,si...","14Rq3sEa,1VTZwwBQ,1xoV23DQ,1f1NhrOS,1XBJ0jUP","iPhone 週邊,Android 週邊,肩背包/斜背包,平板/電腦保護套,其他7","mini,love,cute,萌,悠遊卡手機殼,ipadmini,貓,可愛,悠遊卡保護殼,i...",0.883987,0.116013
4,4,"bang-on-shop,pollinosis,lunablue,nadia,ishan13...","1xfaWs4D,1lwO72OY,1unlj5pu,1ukhY4Bu,1xHzNBML,1...","胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,胸針,項鍊/墜子,髮飾,鑰匙圈/鑰匙包,耳環,...","雙鍊,小物,項鍊,仿珍珠,蕾絲,施華洛世奇水晶,懷錶,創意,design,婚禮,復古手工耳環...",0.591378,0.408622


###Add Browing records

In [89]:
tid=pd.read_csv("Intermediate/tid.csv",sep=",",encoding='utf8')

In [90]:
browse_list=tid['tid']

In [91]:
for i in xrange(len(UserDf)):
    browse_num=np.random.randint(0,5,1)
    if browse_num==0:
         UserDf.ix[i,'Browse']=0
    else:    
        browse_item=list(browse_list[np.random.choice(len(browse_list),browse_num)])
        browse=",".join(browse_item)
        UserDf.ix[i,'Browse']=browse

In [94]:
for i in xrange(len(UserDf)):
    if UserDf.ix[i,'Product Tag'] is np.nan:
        print i
    else:
        continue

In [65]:
UserDf['Product Tag']=UserDf['Product Tag'].fillna(u'0')

In [95]:
UserDf.to_csv("Intermediate/SimUserData.csv",sep=",",encoding='utf8',index=False)